In [1]:
import numpy as np
import torch
from grid2vec import *
import json
import pandas as pd
from utils import Timer, copy_file
from joblib import Parallel, delayed
from traj2grid import Traj2Grid
import traj_dist.distance as tdist
from logging import raiseExceptions
from parameters import *
import numpy as np
import time
# import modin.pandas as pd
# import ray
import fastBPE

timer = Timer()

In [3]:
row_num = 400
dict_path = "data/str_grid2idx_400_44612.json"
str_grid2idx = json.load(open(dict_path))
t2g = Traj2Grid(row_num, column_num, min_lon, min_lat, max_lon, max_lat)
grid2idx = {eval(g): str_grid2idx[g] for g in list(str_grid2idx)}
t2g.set_vocab(grid2idx)
timer.tok(f"load dict{dict_path}")

timer.tik()
for i in range(1, 5):
    df = pd.read_csv(
        f"{data_dir}/full/gps_201611{str(i).zfill(2)}",
        names=["name", "id", "time", "lon", "lat"],
        usecols=["id", "lon", "lat"],
    )  # lon经度 lat纬度
    timer.tok(f"read{str(i).zfill(2)}")
    
    pad = 0.002
    l = len(df)
    df = df[(df["lon"] > 104.04214 + pad) & (df["lon"] < 104.12958 - pad)]
    df = df[(df["lat"] > 30.65294 + pad) & (df["lat"] < 30.72775 - pad)]
    timer.tok(f"剩{len(df)}/{l}个点，筛掉{round(100 - 100 * len(df) / l)}%")
    
    def group_concat(group: pd.DataFrame):
        origin_traj = [((row["lon"]), row["lat"])
                    for index, row in group.iterrows()]
        traj_1d = t2g.convert1d(origin_traj, diff=False)
        series = pd.Series({
            "origin_trajs": origin_traj,
            "trajs": traj_1d,
            "origin_len": len(origin_traj),
            "len": len(traj_1d),
            "max_lon": group["lon"].max(),
            "max_lat": group["lat"].max(),
            "min_lon": group["lon"].min(),
            "min_lat": group["lat"].min(),
        })
        return series


    # group-apply
    res = Parallel(n_jobs=44)(delayed(group_concat)(group)for name, group in df.groupby("id"))
    df = pd.DataFrame(res)
    timer.tok("group-apply")


load dictdata/str_grid2idx_400_44612.json done, 86.798s after tik start
tik start
read01 done, 18.698s after tik start
剩30469437/32155517个点，筛掉5% done, 20.606s after tik start


KeyboardInterrupt: 